# Jupyter Notebooks už jen ve vědě

## Aleš Křenek

### ÚVT MU

## Lenost nade vše

- nebudu vymýšlet nic nového, recykluji existující materiál
- zkušeností s vlatním nepořádkem k tomu mám dost
- aplikace, které z toho vyrostly


## Tato prezentace je také Jupyter Notebook

- můžeme ji prokládat živým kódem

In [ ]:
from decimal import Decimal,getcontext
getcontext().prec=100
Decimal(1).exp()

- není nejkrásnější, přestal mi fungovat můj CSS

### Dockerfile (podstatné části)

```
FROM cerit.io/ljocha/gromacs:2023-2-plumed-2-9-afed-pytorch-model-cv as gmx
FROM quay.io/jupyter/base-notebook

USER 0
RUN apt update && apt install -y ...

RUN pip install nglview mdtraj 
RUN pip install jupyterlab_rise

RUN cd /tmp && git clone --single-branch -b k8s https://github.com/ljocha/GromacsWrapper.git && pip install ./GromacsWrapper

COPY --from=gmx /gromacs /gromacs

USER ${NB_USER}
```

In [ ]:
def _fw(v):
    v.layout.display = 'flex'
    v.layout.width = '100%'
    v.handle_resize()
    return v

In [ ]:
import mdtraj as md
import nglview as nv

## Molekulární dynamika

In [ ]:
tr = md.load('pbc.xtc',top=f'6pxm-box.gro')
_fw(nv.show_mdtraj(tr))

## Standardní simulace proteinu
Podle tutoriálu http://www.mdtutorials.com/gmx/lysozyme/

### 1. Vstupní data
Stáhneme vhodný soubor z veřejné databáze PDB. Pro demo účely poslouží  obvyklý pokusný králík _tryptophan cage_ (1L2Y) -- malý protein s netriviální strukturou.

In [ ]:
import urllib
pdb='1l2y'

In [ ]:
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb}.pdb',f'{pdb}.pdb')
_fw(nv.show_file(f'{pdb}.pdb'))

### 2. Příprava molekuly k simulaci
- zpracování formátu pdb, specifikace modelu solventu a silového pole
- definice simulačního boxu
- přidání solventu

In [ ]:
import gromacs as gmx

In [ ]:
gmx.pdb2gmx(f=pdb+'.pdb',o=pdb+'.gro',water='tip3p',ff='amber99',p=pdb+'.top',ignh=True)

In [ ]:
gmx.editconf(f=pdb+'.gro',o=pdb+'-box.gro',c=True,d='2.5', bt='cubic')

In [ ]:
gmx.solvate(cp=pdb+'-box.gro',cs='spc216.gro',o=pdb+'-solv.gro',p=pdb+'.top')

In [ ]:
v=nv.show_file(f'{pdb}-solv.gro',default_representation=False)
v.add_ball_and_stick('protein')
v.add_cartoon('protein',color='rainbow')
v.add_line('water',color='grey')
_fw(v)

### 3. Přidání iontů
- vytvoření fyziologického roztoku
- vyrovnání náboje proteinu

In [ ]:
with open('ions.mdp','w') as ions: ions.write("""\
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme	= Verlet    ; Buffered neighbor searching 
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = cutoff    ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
""")
    
gmx.grompp(f='ions.mdp',c=pdb+'-solv.gro',p=pdb+'.top',o='ions.tpr')

In [ ]:
gmx.select(s=pdb+'-solv.gro',on='solv.ndx',select='SOL')
gmx.genion(s='ions.tpr',n='solv.ndx',o=pdb+'-ions.gro',p=pdb+'.top',pname='NA',nname='CL',neutral=True,conc=0.15)

In [ ]:
v=nv.show_file(f'{pdb}-ions.gro',default_representation=False)
v.add_ball_and_stick('protein')
v.add_cartoon('protein',color='white')
v.add_line('water',color='grey')
v.add_ball_and_stick("_Na")
v.add_ball_and_stick("_Cl")
_fw(v)

### 4. Minimalizace energie před startem simulace
Uměle vytvořený systém musí být blízko rovnovážného stavu, jinak hrozí nestability. 

In [ ]:
with open('minim.mdp','w') as m: m.write("""\
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.005          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

nstxout                 = 10         
nstvout                 = 0        
nstfout                 = 0
nstenergy               = 10         
""")
gmx.grompp(f='minim.mdp',c=pdb+'-ions.gro',p=pdb+'.top',o='em.tpr')

In [ ]:
gmx.mdrun(deffnm='em')

### 5. Ekvilibrace
- molekula proteinu zůstává fixovaná
- molekulám vody je uměle přiřazena počáteční rychlost
- celý systém se ve dvou krocích postupně "zahřeje" a "natlakuje" do požadovaného stavu

In [ ]:
with open('nvt.mdp','w') as nvt: nvt.write('''\
title                   = OPLS Lysozyme NVT equilibration 
define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = 50000     ; 2 * 50000 = 100 ps
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = no        ; first dynamics run
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is off
pcoupl                  = no        ; no pressure coupling in NVT
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = yes       ; assign velocities from Maxwell distribution
gen_temp                = 300       ; temperature for Maxwell distribution
gen_seed                = -1        ; generate a random seed
''')

In [ ]:
gmx.grompp(f="nvt.mdp",c="em.gro",r="em.gro",p=pdb+'.top',o="nvt.tpr")

In [ ]:
gmx.mdrun(deffnm='nvt')

In [ ]:
with open('npt.mdp','w') as npt: npt.write('''\
define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = 50000     ; 2 * 50000 = 100 ps
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = yes       ; Restarting after NVT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is on
; ljocha pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupl = C-rescale
pcoupltype              = isotropic             ; uniform scaling of box vectors
; ljocha tau_p                   = 2.0                   ; time constant, in ps
tau_p = 5.0
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
refcoord_scaling        = com
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 
''')

In [ ]:
gmx.grompp(f="npt.mdp",c="nvt.gro",r="nvt.gro",p=pdb+'.top',o="npt.tpr")

In [ ]:
gmx.mdrun(deffnm='npt')

Zobrazení výsledných průběhů tlaku, teploty a hustoty

In [ ]:
gmx.energy(f='npt.edr',o='press.xvg',input='Pressure')
gmx.energy(f='npt.edr',o='dens.xvg',input='Density')
gmx.energy(f='npt.edr',o='temp.xvg',input='Temperature')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])
dens = np.loadtxt('dens.xvg',comments=['#','@'])

plt.figure(figsize=(15,9))
plt.subplot(311)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")

plt.subplot(312)
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(dens[:,0],dens[:,1])

plt.subplot(313)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

## Kam s ním?

- vlastní notebook (se samostatnou GPU)
- vlastní server pod stolem
- cloudová služba, nejlépe JupyterHub
  - https://hub.cloud.e-infra.cz/
  - zdroje jsou sdíleny, plýtvání se netoleruje


## Kubernetes, další kontejnery a více zdrojů
- JupyterHub běží v Kubernetes
- pro samotný notebook alokujeme málo
  - 1--2 jádra CPU
  - 2--16 GB RAM (zejména nároky na vizualizaci)
  - bez GPU
- náročné výpočty spouštíme jako další kontejnery
  - alokace zdrojů podle potřeby
  - jen když jsou skutečně třeba
  - sdílený volume, případně sofistikovanější IPC

In [ ]:
mdrunner=gmx.MDrunnerK8s(
    image='cerit.io/ljocha/gromacs:2023-2-plumed-2-9-afed-pytorch-model-cv',
    workdir='europen2024'
)

def k8s_mdrun(mpi=1,omp=2,gpus=1,retry=10,**kwargs):
    mdrunner.run(
        pre={'cores':omp*mpi,'mpi':mpi,'omp':omp,'gpus':gpus, 'retry': retry},
        mdrunargs={**kwargs,'ntomp':omp},
        ncores=mpi
    )

In [ ]:
time = 500 #ps
nsteps = time * 500 # XXX: dt = 0.002

with open('md.mdp','w') as mdp: mdp.write(f'''\
integrator              = md        ; leap-frog integrator
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 0         ; suppress bulky .trr file by specifying 
nstvout                 = 0         ; 0 for output frequency of nstxout,
nstfout                 = 0         ; nstvout, and nstfout
nstenergy               = 5000      ; save energies every 10.0 ps
nstlog                  = 5000      ; update log file every 10.0 ps
nstxout-compressed      = 5000      ; save compressed coordinates every 10.0 ps
compressed-x-grps       = System    ; save the whole system
; Bond parameters
continuation            = yes       ; Restarting after NPT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Neighborsearching
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Dispersion correction
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 
nsteps = {nsteps}
''')
gmx.grompp(f="md.mdp",c="npt.gro",r="npt.gro",p=pdb+'.top',o="md.tpr")

In [ ]:
k8s_mdrun(mpi=1,omp=2,deffnm='md',retry=2)

In [ ]:
gmx.trjconv(f='md.xtc',s='npt.gro',fit='progressive',input='Protein Protein'.split(),o=f'{pdb}-protein.xtc')
gmx.trjconv(f=f'{pdb}-protein.xtc',s=f'{pdb}-box.gro',pbc='nojump',input='Protein',o=f'{pdb}-pbc.xtc')

In [ ]:
tr = md.load(f'{pdb}-pbc.xtc',top=f'{pdb}-box.gro')
_fw(nv.show_mdtraj(tr))

## Bumbrlíčci
- ideje vs. realita
  - původní záměr -- microservices
  - NVidia, AMD, pyTorch, TensorFlow, ... koncept rozbili
  - akademické prostředí nezůstává pozadu
- co s tím
  - není to dobře a někdy to ani nejde
  - oddělujme alespoň velké softwarové balíky
  - využijme prostředky K8s, klidně ad hoc
- režie startu kontejneru
  - rozsáhlejší výpočet ji amortizuje
  - pro krátké rychloobrátkové "sidecar"

https://gitlab.ics.muni.cz/3086/ASMSA/-/blob/master/gmx-wrap2.sh
```
...
kubectl get job/asmsa-gmx 2>/dev/null >/dev/null || kubectl apply -f - >&2 <<EOF
apiVersion: batch/v1
kind: Job
metadata:
  name: asmsa-gmx
spec:
...
      containers:
      - image: cerit.io/ljocha/gromacs:2023-2-plumed-2-9-afed-pytorch-model-cv-2
...
        resources:
          requests:
            cpu: '.1'
            memory: 8Gi
            nvidia.com/gpu: 0
...
EOF
kcmd="cd /mnt/ && gmx ..."
kubectl wait --for=condition=ready pod -l job=asmsa-gmx >&2
kubectl exec job/asmsa-gmx -- bash -c "$kcmd <<<\"$input\""
```

## Služba pro širší komunitu
- typicky průvodní jev publikace
  - učesaný kód
  - obsáhlejší komentáře a návody
- vlastní instance JupyterHubu
  - předepsaný konkrétní image
  - zdroje nastavené tak akorát
  - federovaná autentizace, vzájemná izolace uživatelů
  - https://metadynminer.dyn.cloud.e-infra.cz
  - https://github.com/Jan8be/metadynminer.py
- anonymní spuštění na https://mybinder.org/

## Research in progress
- střídavé fáze úklidu a přirozeného vývoje
- notebooky jako efektivní interdisciplinární komunikační platforma :-)
- https://hub.cloud.e-infra.cz/user/ljocha/asmsa-24-8
- https://gitlab.ics.muni.cz/3086/ASMSA

## GUI

In [ ]:
gmx.energy(f='md.edr',o=f'{pdb}-potential.xvg',input='Potential')
energy = np.loadtxt(f'{pdb}-potential.xvg',comments=['#','@'])

In [ ]:
import ipywidgets as w
import plotly.graph_objects as go

def _frame(e):
    egr.data[1].x = [e.new]
    egr.data[1].y = [energy[e.new,1]]

tr = md.load(f'{pdb}-pbc.xtc',top=f'{pdb}-box.gro')
frame = w.IntSlider(description='Frame',min=0,max=len(tr)-1,layout=w.Layout(width='100%', display='flex'))
frame.observe(_frame,'value')
view = nv.show_mdtraj(tr)
egr = go.FigureWidget(data=[
    go.Scatter(
        x=np.arange(len(tr)),y=energy[:,1],
        marker=go.scatter.Marker(color='blue')
    ),
    go.Scatter(
        x=[0],y=[energy[0,1]],
        marker=go.scatter.Marker(line=dict(width=5),symbol=100,size=15,color='red')
    )
])

def _rep(e):
		view.clear()
		if licorice.value:
			view.add_licorice()
		if cartoon.value:
			view.add_cartoon(color='residueindex')

licorice = w.Checkbox(description='Licorice',value=False)
cartoon = w.Checkbox(description='Cartoon',value=True)
licorice.observe(_rep,'value')
cartoon.observe(_rep,'value')

w.link((frame,'value'), (view,'frame'))
w.VBox([w.HBox([view,w.VBox([licorice,cartoon])]),egr,frame])

- Voila: nadstavba nad Jupyterem, zobrazí jen výstup
- https://gromacs-hub.dyn.cloud.e-infra.cz

## Shrnutí
- od osobního chaosu k otevřené vědě
- efektivní nástroj pro týmovou práci
- nerovnoměrné využití infrastruktury bez plýtvání
- spojitá škála od experimenálního kódu k aplikaci pro koncové uživatele